## Week 5 Final Report
### Finding a good location for  a New Restaurant in Lagos, Nigeria

#### * Scrap data from wikepedia page
#### * Get the geographical coordinates
#### *  Create a map of Lagos with neighborhood
#### * Use the Foursquare API to explore the neighborhoods
#### * Analyze Each Neighborhood
#### * Cluster Neighborhoods
#### * Examine Clusters
#### * Observations


In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library


## Scrap data from wikepedia page

In [4]:
# send the GET request
data = requests.get("https://en.m.wikipedia.org/wiki/Category:Neighborhoods_of_Lagos").text
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [5]:
# create a list to store neighborhood data
neighborhoodList = []
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
    

Create a data frame from the list

In [6]:
lagos_df = pd.DataFrame({"Neighborhood": neighborhoodList})
lagos_df.head()

,Neighborhood
0,Abule Egba
1,Agege
2,Ajegunle
3,Alimosho
4,Apapa


In [8]:
lagos_df.shape

(15, 1)

##  Get the geographical coordinates

In [9]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Lagos, Nigeria'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [10]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in lagos_df["Neighborhood"].tolist() ]

In [11]:
coords

[[6.647500000000036, 3.303060000000073],
 [6.6256100000000515, 3.312620000000038],
 [6.603110000000072, 3.285250000000076],
 [6.609270000000038, 3.255800000000022],
 [6.437950000000058, 3.3643600000000333],
 [6.535000000000025, 3.3947200000000635],
 [6.471940000000075, 3.3791700000000446],
 [6.4538853537020975, 3.3663172074042054],
 [6.456060000000036, 3.4422200000000203],
 [6.4094800000000305, 4.0915200000000596],
 [6.4469400000000405, 3.4152800000000525],
 [6.569470000000024, 3.3952000000000453],
 [6.537850000000049, 3.385340000000042],
 [6.465750000000071, 3.544340000000034],
 [6.454700000000059, 3.3887600000000475]]

Create a dataframe with the Neighborhood and their coordinates

In [12]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
# merge the coordinates into the original dataframe
lagos_df['Latitude'] = df_coords['Latitude']
lagos_df['Longitude'] = df_coords['Longitude']

In [13]:
lagos_df.shape

(15, 3)

In [14]:
lagos_df

,Neighborhood,Latitude,Longitude
0,Abule Egba,6.647500,3.303060
1,Agege,6.625610,3.312620
2,Ajegunle,6.603110,3.285250
3,Alimosho,6.609270,3.255800
4,Apapa,6.437950,3.364360
5,Bariga,6.535000,3.394720
6,Iddo Island,6.471940,3.379170
7,"Ijora, Lagos",6.453885,3.366317
8,Ikoyi,6.456060,3.442220
9,Lekki,6.409480,4.091520


In [15]:
# save the DataFrame as CSV file
lagos_df.to_csv("lagos_df.csv", index=False)

## Create a map of Lagos with neighborhood

In [17]:
# get the coordinates of Lagos
address = 'Lagos, Nigeria'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Lagos, Nigeria {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Lagos, Nigeria 6.4550575, 3.3941795.


In [19]:
# create map of Lagos using latitude and longitude values
map_lagos = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(lagos_df['Latitude'], lagos_df['Longitude'], lagos_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_lagos)  
    
map_lagos

In [20]:
# save the map as HTML file
map_lagos.save('map_lagos.html')

## Use the Foursquare API to explore the neighborhoods

In [1]:
# define Foursquare Credentials and Version
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = ''# your Foursquare Secret
VERSION = '20200627' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


 Get the top 100 venues that are within a radius of 2000 meters.

In [22]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(lagos_df['Latitude'], lagos_df['Longitude'], lagos_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [23]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'Venue_name', 'Venue_latitude', 'Venue_longitude', 'Venue_category']

print(venues_df.shape)
venues_df.head()

(238, 7)


,Neighborhood,Latitude,Longitude,Venue_name,Venue_latitude,Venue_longitude,Venue_category
0,Abule Egba,6.6475,3.30306,Kotongora Market,6.648938,3.305505,Flea Market
1,Abule Egba,6.6475,3.30306,Mr biggs abule egba,6.649176,3.305269,Burger Joint
2,Abule Egba,6.6475,3.30306,otunba,6.648647,3.306266,Hookah Bar
3,Abule Egba,6.6475,3.30306,"Isokoko Market, Agege",6.658001,3.300299,Farmers Market
4,Abule Egba,6.6475,3.30306,Fagba Pipeline,6.655874,3.317279,Light Rail Station


In [24]:
#check how many venues were returned for each neighorhood
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,Venue_name,Venue_latitude,Venue_longitude,Venue_category
Neighborhood,,,,,,
Abule Egba,5,5,5,5,5,5
Agege,9,9,9,9,9,9
Ajegunle,10,10,10,10,10,10
Alimosho,4,4,4,4,4,4
Apapa,8,8,8,8,8,8
Bariga,8,8,8,8,8,8
Iddo Island,15,15,15,15,15,15
"Ijora, Lagos",10,10,10,10,10,10
Ikoyi,29,29,29,29,29,29


In [26]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(venues_df['Venue_category'].unique())))

There are 85 uniques categories.


In [28]:
# print out the list of categories
venues_df['Venue_category'].unique()

array(['Flea Market', 'Burger Joint', 'Hookah Bar', 'Farmers Market',
       'Light Rail Station', 'Fast Food Restaurant', 'Campground',
       'Bus Station', 'Market', 'Gym', 'Bus Stop', 'Pizza Place', 'Hotel',
       'Boutique', 'Park', 'Basketball Court', 'Department Store', 'Bank',
       'Bar', 'Airport', 'Shopping Mall', 'Playground',
       'Indian Restaurant', 'Plaza', 'Convenience Store', 'Restaurant',
       'River', 'Pharmacy', 'Art Gallery', 'Train Station',
       'Liquor Store', 'Rental Car Location', 'Noodle House',
       'Fish Market', 'Lounge', 'Neighborhood', 'Gym / Fitness Center',
       'Athletics & Sports', 'Diner', 'Golf Course', 'Wine Bar',
       'American Restaurant', 'Ice Cream Shop', 'Café', 'Sports Bar',
       'Bistro', 'Spa', 'Deli / Bodega', 'Pool Hall', 'Arcade',
       'Sports Club', 'Nightclub', 'Music Venue', 'Grocery Store',
       'Mediterranean Restaurant', 'Cocktail Bar', 'Convention Center',
       'Breakfast Spot', 'Brazilian Restaurant',
    

## Analyze Each Neighborhood

In [30]:
# one hot encoding
lagos_onehot = pd.get_dummies(venues_df[['Venue_category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
lagos_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [lagos_onehot.columns[-1]] + list(lagos_onehot.columns[:-1])
lagos_onehot = lagos_onehot[fixed_columns]

print(lagos_onehot.shape)
lagos_onehot.head()

(238, 86)


,Neighborhoods,African Restaurant,Airport,American Restaurant,Arcade,Art Gallery,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,Bank,Bar,Baseball Stadium,Basketball Court,Beer Garden,Bistro,Boutique,Brazilian Restaurant,Breakfast Spot,Burger Joint,Bus Station,Bus Stop,Café,Campground,Chinese Restaurant,Cocktail Bar,Convenience Store,Convention Center,Deli / Bodega,Department Store,Dessert Shop,Diner,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Fried Chicken Joint,Furniture / Home Store,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hookah Bar,Hotel,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Modern European Restaurant,Music Venue,Neighborhood,Nightclub,Noodle House,Park,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Pool Hall,Rental Car Location,Resort,Restaurant,River,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Spa,Sports Bar,Sports Club,Stables,Stadium,Supermarket,Sushi Restaurant,Train Station,Wine Bar,Wine Shop
0,Abule Egba,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Abule Egba,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Abule Egba,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Abule Egba,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Abule Egba,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [31]:
lagos_grouped = lagos_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(lagos_grouped.shape)
lagos_grouped

(15, 86)


,Neighborhoods,African Restaurant,Airport,American Restaurant,Arcade,Art Gallery,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,Bank,Bar,Baseball Stadium,Basketball Court,Beer Garden,Bistro,Boutique,Brazilian Restaurant,Breakfast Spot,Burger Joint,Bus Station,Bus Stop,Café,Campground,Chinese Restaurant,Cocktail Bar,Convenience Store,Convention Center,Deli / Bodega,Department Store,Dessert Shop,Diner,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Fried Chicken Joint,Furniture / Home Store,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hookah Bar,Hotel,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Modern European Restaurant,Music Venue,Neighborhood,Nightclub,Noodle House,Park,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Pool Hall,Rental Car Location,Resort,Restaurant,River,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Spa,Sports Bar,Sports Club,Stables,Stadium,Supermarket,Sushi Restaurant,Train Station,Wine Bar,Wine Shop
0,Abule Egba,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.20,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Agege,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.222222,0.111111,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Ajegunle,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.100,0.000000,0.00,0.1,0.000000,0.000000,0.1,0.000000,0.000000,0.00,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Alimosho,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.250000,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

In [32]:
len(lagos_grouped[lagos_grouped["Restaurant"] > 0])

3

 Create a new DataFrame for Restaurant data only

In [33]:
lagos_restau = lagos_grouped[["Neighborhoods","Restaurant"]]

In [34]:
lagos_restau

,Neighborhoods,Restaurant
0,Abule Egba,0.000000
1,Agege,0.000000
2,Ajegunle,0.000000
3,Alimosho,0.000000
4,Apapa,0.000000
5,Bariga,0.125000
6,Iddo Island,0.000000
7,"Ijora, Lagos",0.000000
8,Ikoyi,0.068966
9,Lekki,0.000000


## Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Lagos into 3 clusters.

In [45]:
lclusters = 3

lagos_clustering = lagos_restau.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=lclusters, random_state=0).fit(lagos_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 1, 0, 0, 2, 0])

In [46]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
lagos_merged = lagos_restau.copy()

# add clustering labels
lagos_merged["Cluster Labels"] = kmeans.labels_

In [47]:
lagos_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
lagos_merged

,Neighborhood,Restaurant,Cluster Labels
0,Abule Egba,0.000000,0
1,Agege,0.000000,0
2,Ajegunle,0.000000,0
3,Alimosho,0.000000,0
4,Apapa,0.000000,0
5,Bariga,0.125000,1
6,Iddo Island,0.000000,0
7,"Ijora, Lagos",0.000000,0
8,Ikoyi,0.068966,2
9,Lekki,0.000000,0


In [49]:
# merge lagos_grouped with lagos_data to add latitude/longitude for each neighborhood
lagos_merged = lagos_merged.join(lagos_df.set_index("Neighborhood"), on="Neighborhood")

print(lagos_merged.shape)
lagos_merged.head() # check the last columns!

(15, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Abule Egba,0.0,0,6.64750,3.30306
1,Agege,0.0,0,6.62561,3.31262
2,Ajegunle,0.0,0,6.60311,3.28525
3,Alimosho,0.0,0,6.60927,3.25580
4,Apapa,0.0,0,6.43795,3.36436


In [50]:
# sort the results by Cluster Labels
print(lagos_merged.shape)
lagos_merged.sort_values(["Cluster Labels"], inplace=True)
lagos_merged

(15, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Abule Egba,0.000000,0,6.647500,3.303060
1,Agege,0.000000,0,6.625610,3.312620
2,Ajegunle,0.000000,0,6.603110,3.285250
3,Alimosho,0.000000,0,6.609270,3.255800
4,Apapa,0.000000,0,6.437950,3.364360
6,Iddo Island,0.000000,0,6.471940,3.379170
7,"Ijora, Lagos",0.000000,0,6.453885,3.366317
9,Lekki,0.000000,0,6.409480,4.091520
10,Obalende,0.000000,0,6.446940,3.415280
12,Somolu,0.000000,0,6.537850,3.385340


In [53]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(lclusters)
ys = [i+x+(i*x)**2 for i in range(lclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(lagos_merged['Latitude'], lagos_merged['Longitude'], lagos_merged['Neighborhood'], lagos_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [54]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## Examine Clusters

Cluster 0

In [55]:
lagos_merged.loc[lagos_merged['Cluster Labels'] == 0]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Abule Egba,0.0,0,6.647500,3.303060
1,Agege,0.0,0,6.625610,3.312620
2,Ajegunle,0.0,0,6.603110,3.285250
3,Alimosho,0.0,0,6.609270,3.255800
4,Apapa,0.0,0,6.437950,3.364360
6,Iddo Island,0.0,0,6.471940,3.379170
7,"Ijora, Lagos",0.0,0,6.453885,3.366317
9,Lekki,0.0,0,6.409480,4.091520
10,Obalende,0.0,0,6.446940,3.415280
12,Somolu,0.0,0,6.537850,3.385340


Cluster 1

In [56]:
lagos_merged.loc[lagos_merged['Cluster Labels'] == 1]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
5,Bariga,0.125000,1,6.53500,3.39472
11,Ogudu,0.142857,1,6.56947,3.39520


In [57]:
lagos_merged.loc[lagos_merged['Cluster Labels'] == 2]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
8,Ikoyi,0.068966,2,6.45606,3.44222


## OBSERVATIONS
From the analysis above, it shows that there are a lot of restaurant in cluster 1 and 2 (Bariga, Ogudu and Ikoyi). whereas there are very few restaurants in cluster 0. Any neighborhood in cluster 0 will make a good enviroment to locate a restauarant as there are very little competitions. This project recommends property developers to consider opening a restaurant in the neighborhoods of cluster 0. 